In [0]:
import json
import requests
import urllib

# 0. Preliminary step to get sample data

This preliminary step is reproducing Lorella's workflow Python file:
https://i-lab.public.data.uu.nl/vault-ocex/ChroniclItaly%20-%20Italian%20American%20newspapers%20corpus%20from%201898%20to%201920%5B1529330521%5D/original/
I just added a folder "data_1" to keep all files in one folder

In [0]:
mkdir 'data1'

In [0]:
# Base URL
chronam = 'https://chroniclingamerica.loc.gov/'

# Chronicling America search results
results = 'https://chroniclingamerica.loc.gov/search/pages/results/?date1=1880&date2=1920&searchType=advanced&language=ita&sequence=1&lccn=2012271201&lccn=sn85066408&lccn=sn85055164&lccn=sn85054967&lccn=sn88064299&lccn=sn84037024&lccn=sn84037025&lccn=sn86092310&proxdistance=5&state=California&state=District+of+Columbia&state=Massachusetts&state=Pennsylvania&state=Piedmont&state=Vermont&state=West+Virginia&rows=100&ortext=&proxtext=&phrasetext=&andtext=&dateFilterType=yearRange&page=11&sort=date'

# Count to keep track of downloaded files
count = 0

# Gets search results in JSON format
results_json = results + '&format=json'


In [0]:
# Returns JSON 
def get_json(url):
    data = requests.get(url)
    return(json.loads(data.content))
    
data = get_json(results_json)

In [0]:
# Cycle through JSON results
for page in data['items']:
    # Create URL
    hit = str(page['id'])
    seed = hit + 'ocr.txt'
    download_url = chronam + seed
 
    # Create file name
    file_name = download_url.replace('/', '_')
    file_name = 'data1/' + file_name[41:]
    
    # Download .txt of the page
    urllib.request.urlretrieve(download_url, str(file_name))
    count += 1

# 1. Data preparation

## 1.1. Grouping all texts files
A dataframe is first created to keep individual files at their initial state, and the name of each file

In [0]:
import os
import pandas as pd

In [0]:
#list of the file names
files_list = os.listdir('data1')

In [0]:
#insert file names into a df
sources = pd.DataFrame(files_list, columns=['file_name'])

In [0]:
#function to read the content of the text files
def readTxtContent(fileName):
  with open('data1/' + fileName, 'r') as file:
    return ' ' + file.read().replace('\n', ' ') + ' '

In [0]:
# adding a column to the dataframe containing file content
sources['file_content'] = sources['file_name'].apply(lambda x: readTxtContent(x))

In [11]:
# for verification purposes later, count the nr of characters for each content
sources['file_len'] = sources['file_content'].apply(lambda x: len(x))
sources['file_len'].sum()

1779770

In [0]:
# variable containing all texts together
corpus = ''
for i in range(len(sources)):
  corpus += sources['file_content'][i]

In [13]:
# check length
len(corpus)

1779770

## 1.2 Removing stop words

In [0]:
#lower case the corpus
corpus_low = corpus.lower()

In [0]:
import spacy
from spacy.tokenizer import Tokenizer

#setting up for the language
from spacy.lang.it import Italian
nlp = Italian()

In [0]:
tokenizer = Tokenizer(nlp.vocab)

In [0]:
tokens = tokenizer(corpus_low)

In [62]:
len(tokens)

298045

Below are a few options examples (only run the cell that fits your needs or compose your own solution) 


In [0]:
#lower case
tokens_low = [w.text.lower() for w in tokens]

In [0]:
# testing lemmatized tokens
#[w.lemma_ for w in tokens]

In [0]:
# lower case and remove punctuation
tokens_low_punct = [w.text.lower() for w in tokens if w.text.isalnum()]

In [0]:
# lower case, remove punctuation and only keep words that have more than 3 letters
tokens_low_long = [w.text.lower() for w in tokens if (w.text.isalnum() and len(w.text) > 3 )]

Stop words

In [0]:
from spacy.lang.it.stop_words import STOP_WORDS

In [0]:
# show list of default Italian stopwords
# STOP_WORDS

In [0]:
# add custom stop word
ital_stopwords = STOP_WORDS
# to append list of words added by user: ital_stopwords.extend(user_input)
# to remove words: ital_stopwords.remove(user_input)

In [0]:
# "stopwords.words('italian')" can be replaced by a custom list input by the user
tokenized_corpus_without_sw = [w for w in tokens_low_long if not w in ital_stopwords]

## 1.3 Stem 

In [0]:
# Stemming is available via NLTK and not Spacy

In [0]:
import nltk
from nltk.stem.snowball import SnowballStemmer

In [0]:
#initialize with needed language
stemmer = SnowballStemmer("italian")

In [0]:
stemmed_corpus = [stemmer.stem(w) for w in tokenized_corpus_without_sw]

## 1.4 Lemmatize

In [0]:
#Lemmatize is available in multiple languages in Spacy and not in NLTK (only English)

In [0]:
# this cell will be deleted later
#lemmatize (English only)
#from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
#wnl = WordNetLemmatizer()
#lemmatized_corpus = [wnl.lemmatize(w) for w in tokenized_corpus_without_sw]

In [0]:
from spacy.lemmatizer import Lemmatizer

In [0]:
for word in sentence6:
    print(word.text,  word.lemma_)

In [0]:
# create dataframe with corpus
lemmatized_corpus = pd.DataFrame(tokenized_corpus_without_sw, columns=['tokens'])

In [0]:
# lemmatized_corpus['lemmatized_token'] = lemmatized_corpus['tokens'].apply(lambda x: x.lemma_)